In [0]:
!pip install --upgrade torch-scatter
!pip install --upgrade torch-sparse
!pip install --upgrade torch-cluster
!pip install --upgrade torch-spline-conv 
!pip install torch-geometric

  Created wheel for torch-scatter: filename=torch_scatter-1.4.0-cp36-cp36m-linux_x86_64.whl size=3170441 sha256=fecf1f86e379f9d4cd52436e671518bad96ffc93ca2e49431a949128e4175e29
  Stored in directory: /root/.cache/pip/wheels/25/00/c4/1637b4b3003f29092f4fe2ad4b40dd10906269c1ac2dc82941
Successfully built torch-scatter
  Created wheel for torch-sparse: filename=torch_sparse-0.4.3-cp36-cp36m-linux_x86_64.whl size=3966626 sha256=1ae3488a2555d84fdfe305d6404084a91960e88820cc96b7903c4015dc609b2c
  Stored in directory: /root/.cache/pip/wheels/02/66/2b/befece01c2516f9fb3e7b4d150bb2b871221c73657c9cd7735
Successfully built torch-sparse
  Created wheel for torch-cluster: filename=torch_cluster-1.4.5-cp36-cp36m-linux_x86_64.whl size=16232002 sha256=4e4eab14830e1d6f80f1ddb90ce4652c9a886d7a5da65d37a4ed5b3572bcd99b
  Stored in directory: /root/.cache/pip/wheels/0a/26/7e/a6d6a80eae5ca39b92bc77773f36cf433d5085de18014382b1
Successfully built torch-cluster
  Created wheel for torch-spline-conv: filename=tor

In [0]:
import torch
from torch_geometric.data import Data

In [0]:
x = torch.tensor([[2,1], [5,6], [3,7], [12,0]], dtype=torch.float)
y = torch.tensor([0,1,0,1], dtype=torch.float)

edge_index = torch.tensor([[0, 1, 2, 0, 3],
                           [1, 0, 1, 3, 2]], dtype=torch.long)

In [0]:
data =Data(x=x, edge_index=edge_index)
data

Data(edge_index=[2, 5], x=[4, 2])

In [0]:
data.edge_index

tensor([[0, 1, 2, 0, 3],
        [1, 0, 1, 3, 2]])

In [0]:
device = torch.device('cuda')
data = data.to(device)

In [0]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')

Extracting /tmp/ENZYMES/ENZYMES.zip
Processing...
Done!


In [0]:
len(dataset)

600

In [0]:
data = dataset[0]
data

Data(edge_index=[2, 168], x=[37, 3], y=[1])

In [0]:
data.is_undirected()

True

In [0]:
dataset= dataset.shuffle()
train_daataset = dataset[:540]
test_dataset = dataset[540:]

In [0]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [0]:
print(dataset.num_classes)
print(dataset.num_node_features)


7
1433


In [0]:
dataset[0].edge_index

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])

In [0]:
from torch_geometric.data import DataLoader

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch in loader:
  print(batch)

Batch(batch=[1051], edge_index=[2, 3950], x=[1051, 21], y=[32])
Batch(batch=[931], edge_index=[2, 3724], x=[931, 21], y=[32])
Batch(batch=[1027], edge_index=[2, 3902], x=[1027, 21], y=[32])
Batch(batch=[1170], edge_index=[2, 4180], x=[1170, 21], y=[32])
Batch(batch=[928], edge_index=[2, 3558], x=[928, 21], y=[32])
Batch(batch=[1028], edge_index=[2, 3946], x=[1028, 21], y=[32])
Batch(batch=[1018], edge_index=[2, 3600], x=[1018, 21], y=[32])
Batch(batch=[1025], edge_index=[2, 4086], x=[1025, 21], y=[32])
Batch(batch=[1067], edge_index=[2, 4156], x=[1067, 21], y=[32])
Batch(batch=[1161], edge_index=[2, 4108], x=[1161, 21], y=[32])
Batch(batch=[1068], edge_index=[2, 4042], x=[1068, 21], y=[32])
Batch(batch=[1053], edge_index=[2, 4250], x=[1053, 21], y=[32])
Batch(batch=[1151], edge_index=[2, 4214], x=[1151, 21], y=[32])
Batch(batch=[1041], edge_index=[2, 4134], x=[1041, 21], y=[32])
Batch(batch=[1024], edge_index=[2, 3906], x=[1024, 21], y=[32])
Batch(batch=[994], edge_index=[2, 3722], x=[

In [0]:
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'],
                    pre_transform=T.KNNGraph(k=6),
                    transform=T.RandomTranslate(0.01))

dataset[0]

Extracting /tmp/ShapeNet/raw/train_data.zip
Extracting /tmp/ShapeNet/raw/train_label.zip
Extracting /tmp/ShapeNet/raw/val_data.zip
Extracting /tmp/ShapeNet/raw/val_label.zip
Extracting /tmp/ShapeNet/raw/test_data.zip
Extracting /tmp/ShapeNet/raw/test_label.zip
Processing...
Done!


Data(category=[1], edge_index=[2, 15108], pos=[2518, 3], y=[2518])

In [0]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [0]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [0]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)

data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
  optimizer.zero_grad()
  out = model(data)
  loss = F.nll_loss(input=out[data.train_mask], target=data.y[data.train_mask])
  loss.backward()
  optimizer.step()

In [0]:
model.eval()
_,pred= model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.8030
